In [ ]:
import numpy as np
import pandas as pd
import talib as ta
import tushare as ts
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
mpl.rcParams['axes.unicode_minus']=False #确保‘-’号显示正常
mpl.rcParams['font.sans-serif'] = ['SimHei'] #确保中文显示正常

stock_index = ts.get_k_data('hs300', '2016-01-01', '2017-06-30')
stock_index.set_index('date', inplace=True)
stock_index.sort_index(inplace = True)

stock_index['sma'] = ta.SMA(np.asarray(stock_index['close']), 5) #计算SMA
stock_index['cci'] = ta.CCI(np.asarray(stock_index['high']), np.asarray(stock_index['low']), np.asarray(stock_index['close']), timeperiod=20) #计算CCI

plt.subplot(2,1,1)
plt.title('沪深300 sma cci指标图')
plt.gca().axes.get_xaxis().set_visible(False)
stock_index['close'].plot(figsize = (10,8))
stock_index['sma'].plot(figsize=(10,8))
plt.legend()
plt.subplot(2,1,2)
stock_index['cci'].plot(figsize = (10,8))
plt.legend()
plt.show()

# 产生开仓信号时应使用昨日及前日数据，以避免未来数据
stock_index['yes_close'] = stock_index['close'].shift(1)
stock_index['yes_sma'] = stock_index['sma'].shift(1)
stock_index['yes_cci'] = stock_index['cci'].shift(1)
stock_index['daybeforeyes_close'] = stock_index['close'].shift(2)
stock_index['daybeforeyes_sma'] = stock_index['sma'].shift(2)

stock_index['sma_signal'] = np.where(np.logical_and(stock_index['daybeforeyes_close']<stock_index['daybeforeyes_sma'], stock_index['yes_close']>stock_index['yes_sma']), 1, 0)
stock_index['sma_signal'] = np.where(np.logical_and(stock_index['daybeforeyes_close']>stock_index['daybeforeyes_sma'], stock_index['yes_close']<stock_index['yes_sma']), -1, stock_index['sma_signal'])
stock_index['cci_filter'] = np.where(stock_index['yes_cci'] < -100, 1, 0)
stock_index['cci_filter']  = np.where(stock_index['yes_cci'] > 100,-1, stock_index['cci_filter'])
stock_index['filtered_signal'] = np.where(stock_index['sma_signal']+stock_index['cci_filter']==2, 1, 0) #股价上穿SMA且CCI<-100表明在超卖情况下股价上穿SMA，做多
stock_index['filtered_signal'] = np.where(stock_index['sma_signal']+stock_index['cci_filter']==-2, -1,stock_index['filtered_signal']) #股价下穿SMA且CCI>100表明在超买情况下股价下穿SMA，做空

# 绘制cci，sma指标图
plt.subplot(3,1,1)
plt.title('沪深300 CCI SMA指标图')
plt.gca().axes.get_xaxis().set_visible(False)
stock_index['close'].plot(figsize=(12,8))
stock_index['sma'].plot()
plt.legend(loc='upper left')
plt.subplot(3,1,2)
stock_index['cci'].plot(figsize=(12, 8))
plt.legend(loc='upper left')
plt.subplot(3,1,3)
stock_index['filtered_signal'].plot(figsize=(12, 8), marker='o', linestyle='')
plt.legend(loc='upper left')
plt.show()

#循环法回测
position = 0
for i, item in stock_index.iterrows():
    if item['filtered_signal'] == 1:
        position = 1
    elif item['filtered_signal'] == -1:
        position = -1
    else:
        pass
    stock_index.loc[i, 'position'] = position

plt.subplot(3, 1, 1)
plt.title('600030 CCI持仓图')
plt.gca().axes.get_xaxis().set_visible(False)
stock_index['close'].plot(figsize = (12,12))
plt.legend()
plt.subplot(3, 1, 2)
stock_index['cci'].plot(figsize = (12,12))
plt.legend()
plt.gca().axes.get_xaxis().set_visible(False)
plt.subplot(3, 1, 3)
stock_index['position'].plot(marker='o', figsize=(12,12),linestyle='')
plt.legend()
plt.show()

stock_index['pct_change'] = stock_index['close'].pct_change() #股票每日收益率
stock_index['strategy_return'] = stock_index['pct_change'] * stock_index['position'] #策略每日收益率
stock_index['return'] = (stock_index['pct_change']+1).cumprod() #股票累积收益率
stock_index['strategy_cum_return'] = (1 + stock_index['strategy_return']).cumprod() #策略累积收益率

stock_index[['return', 'strategy_cum_return']].plot(figsize = (12,6)) #股票累积收益率和策略累积收益率绘图
plt.title('600030 CCI收益图')
plt.legend()
plt.show()